In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

##############################################################
pd_crime = pd.read_csv('Crime_Data_from_2020_to_Present_20240701.csv')
pd_crime['year'] = pd_crime['Date Rptd'].apply(lambda x: x[6:10])
pd_crime['month'] = pd_crime['Date Rptd'].apply(lambda x: x[:2])
pd_crime['hour'] = pd_crime['TIME OCC'].apply(lambda x: x // 100)
pd_crime['DATE OCC'] = pd.to_datetime(pd_crime['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')
pd_crime['DoW'] = pd_crime['DATE OCC'].dt.dayofweek
##############################################################
selected_cols = ['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA NAME', 'Vict Age',
       'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc', 'Crm Cd Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'LOCATION',
       'Cross Street', 'LAT', 'LON', 'year', 'month', 'hour', 'DoW']
##############################################################
pd_crime = pd_crime[selected_cols]
pd_crime = pd_crime.query("year < '2024'")
##############################################################
# source: https://fred.stlouisfed.org/series/CALOSA7POP
pd_population = pd.DataFrame(
    {
        'year': ['2020', '2021', '2022', '2023'],
        'population': [9992813, 9809462, 9719765, 9663345]
    }
)


In [ ]:
pd_cr = pd_crime.groupby('year').size().reset_index().merge(pd_population, on='year', how='left')
pd_cr.columns = ['year', 'crime_cnt', 'population']
pd_cr['crime_rate'] = pd_cr['crime_cnt'] / pd_cr['population']

In [ ]:
unemploy_data = {
    "year": ['2020', '2021', '2022', '2023'],
    "unemployment_rate": [12.3, 9.0, 5.0, 5.0]
}

pd_unemployment = pd.DataFrame(unemploy_data)

income_data = {
    "year": ['2020', '2021', '2022'],
    "median_income": [84737, 87562, 83411]
}

pd_income = pd.DataFrame(income_data)
pd_indicators = pd_cr.merge(pd_income, on='year', how='left')\
    .merge(pd_unemployment, on='year', how='left')


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=pd_indicators['year'], y=pd_indicators['unemployment_rate']/100, name="Unemployment Rate", mode='lines+markers'),
)

fig.add_trace(
    go.Scatter(x=pd_indicators['year'], y=pd_indicators['crime_rate'], name="Crime Rate", mode='lines+markers', yaxis="y2")
)

fig.update_layout(
    width=1100, height=450,
    title="<b>Crime Rate and Unemployment Rate From 2020 to 2023</b>",
    xaxis_title="Year",
    yaxis=dict(
        title="Unemployment Rate"
    ),
    yaxis2=dict(
        title="Crime Rate",
        overlaying="y",
        side="right"
    )
)

# Display the figure
fig.write_image("image/unemployment.png")
fig.show()


In [ ]:

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=pd_indicators['year'].astype(int), y=pd_indicators['median_income'], name='Median Income'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=pd_indicators['year'].astype(int), y=pd_indicators['crime_rate'], name='Crime Rate', mode='lines+markers'),
    secondary_y=True,
)

fig.update_layout(
    width=1100, height=450,
    title_text="<b>Crime Rate and Median Household Income from 2020 to 2023</b>",
    title_font=dict(color='black', weight='bold'), 
    xaxis_title="",
    xaxis=dict(
        tickmode='array',
        tickvals=pd_indicators['year'].astype(int),  
        ticktext=pd_indicators['year']
    ),
    annotations=[
        dict(
            x=2023,  
            y=80000,
            xref="x",
            yref="y",
            text="Data for Median Income in 2023 is not available",
            showarrow=True,
            font=dict(
                family="Courier New, monospace",
                size=10,
                color="#ffffff"
            ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=20,
            ay=-30,
            bordercolor="purple",  
            borderwidth=2,
            borderpad=4,
            bgcolor="mediumpurple", 
            opacity=0.8
        )
    ]
)

fig.update_yaxes(title_text="Median Income", secondary_y=False)
fig.update_yaxes(title_text="Crime Rate", secondary_y=True)

fig.update_yaxes(range=[70000, 90000], secondary_y=False)
fig.show()
fig.write_image('image/income.png')

In [ ]:
years = pd_cr['year']
population = pd_cr['population']
crime_rate = pd_cr['crime_rate']


fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=years, y=population, name='Population'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=years, y=crime_rate, name='Crime Rate', mode='lines+markers'),
    secondary_y=True,
)

fig.update_layout(
    title_text="<b>Population and Crime Rate Trends from 2020 to 2023</b>", 
    title_font=dict(color='black', weight='bold'), 
    xaxis_title="Year",
)


fig.update_yaxes(title_text="Population", secondary_y=False)
fig.update_yaxes(title_text="Crime Rate", secondary_y=True)

fig.update_yaxes(range=[9600000, 10000000], secondary_y=False)
fig.show()
fig.write_image('image/population_crime_rate.png')

# Significance test for crime rate over time

In [ ]:
# Significant Test for Crime Rate
import statsmodels.api as sm

X = pd_cr[['year']].astype(int) 
X = sm.add_constant(X) 
Y = pd_cr['crime_rate']  

model = sm.OLS(Y, X).fit()
model_summary = model.summary()
model_summary


# Temporal

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import calendar

month_labels = {f"{i:02d}": calendar.month_abbr[i] for i in range(1, 13)}

monthly_counts = pd_crime['month'].value_counts().sort_index()
hourly_counts = pd_crime['hour'].value_counts().sort_index()
dayofweek_counts = pd_crime['DoW'].value_counts().sort_index()

average_monthly_count = monthly_counts.mean()
average_hourly_count = hourly_counts.mean()
average_dow_count = dayofweek_counts.mean()

fig_month = px.bar(monthly_counts, x=monthly_counts.index.map(month_labels), y=monthly_counts.values,
                   labels={'y': 'Crime Reports', 'x': ''},
                   title="<b>Crime Reports by Month</b>",
                   text=monthly_counts.values)
fig_month.update_yaxes(range=[60000, 78000], showticklabels=True)
fig_month.update_traces(texttemplate='%{text}', textposition='outside')
fig_month.add_trace(go.Scatter(x=monthly_counts.index.map(month_labels), 
                               y=[average_monthly_count]*len(monthly_counts),
                               mode='lines', line=dict(dash='dash', color='blue'),
                               name='Avg. Reports Monthly'))
fig_month.update_layout(legend=dict(x=0.02, y=0.95, bgcolor='rgba(255,255,255,0.5)'))
fig_month.write_image('image/monthly.png')
#fig_month.show()

fig_hour = px.bar(hourly_counts, x=hourly_counts.index.astype(int), y=hourly_counts.values,
                  labels={'y': 'Crime Reports', 'x': ''},
                  title="<b>Crime Reports by Hour</b>",
                  text=hourly_counts.values)
fig_hour.update_xaxes(tickmode='linear', tick0=0, dtick=1, showticklabels=True)
fig_hour.update_traces(marker_color='rgba(255, 179, 186, 1)', texttemplate='%{text}', textposition='outside')
fig_hour.add_trace(go.Scatter(x=hourly_counts.index, y=[average_hourly_count]*len(hourly_counts),
                              mode='lines', line=dict(dash='dash', color='red'),
                              name='Avg. Reports Hourly'))
fig_hour.update_layout(legend=dict(x=0.02, y=0.95, bgcolor='rgba(255,255,255,0.5)'))
fig_hour.write_image('image/hourly.png')
#fig_hour.show()

dow_labels = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
fig_dow = px.bar(dayofweek_counts, x=dayofweek_counts.index.map(dow_labels),
                 y=dayofweek_counts.values, labels={'y': 'Crime Reports', 'x': ''},
                 title="<b>Crime Reports by Day of Week</b>",
                 text=dayofweek_counts.values)
fig_dow.update_traces(marker_color='rgba(156, 217, 107, 1)', texttemplate='%{text}', textposition='outside')
fig_dow.update_yaxes(range=[100000, 140000], showticklabels=True)
fig_dow.add_trace(go.Scatter(x=dayofweek_counts.index.map(dow_labels), 
                             y=[average_dow_count]*len(dayofweek_counts),
                             mode='lines', line=dict(dash='dash', color='green'),
                             name='Avg. Reports DoW'))
fig_dow.update_layout(legend=dict(x=0.02, y=0.95, bgcolor='rgba(255,255,255,0.5)'))
fig_dow.write_image('image/dow.png')
#fig_dow.show()


In [ ]:
heatmap_data = pd_crime.pivot_table(index='hour', columns='DoW', values='DR_NO', aggfunc='count').fillna(0).astype(int)

fig = px.imshow(heatmap_data,
                labels=dict(x="", y="", color="Crime Reports"),
                y=[i for i in range(24)],
                x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                text_auto=True, 
                aspect='equal',  
                color_continuous_scale='Blues',  
                title="<b>Heatmap of Crime Reports by Hour and Day of Week</b>",
                width=800,  
                height=600) 

fig.update_yaxes(tickmode='linear', tick0=0, dtick=1, showticklabels=True)
fig.update_xaxes(side="bottom")  
fig.write_image('image/heatmap_hourly_dow.png')
fig.show()


# Location

In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import numpy as np


filtered_df = pd_crime[
    (pd_crime['LAT'] > 33.5) & (pd_crime['LAT'] < 34.8) &
    (pd_crime['LON'] > -119) & (pd_crime['LON'] < -117.5)
]

lat_step, lon_step = 0.15, 0.15  

filtered_df['lat_cluster'] = np.round(filtered_df['LAT'] / lat_step) * lat_step
filtered_df['lon_cluster'] = np.round(filtered_df['LON'] / lon_step) * lon_step

cluster_summary = filtered_df.groupby(['lat_cluster', 'lon_cluster']).size().reset_index(name='count')

map_la = folium.Map(location=[34.0522, -118.2437], zoom_start=10)


def get_color(count):
    if count < 10000:  
        return 'green'
    elif count < 80000:  
        return 'orange'
    else:
        return 'red'

marker_cluster = MarkerCluster().add_to(map_la)
for idx, row in cluster_summary.iterrows():
    folium.CircleMarker(
        location=[row['lat_cluster'], row['lon_cluster']],
        radius=5 + (row['count'] ** 0.3),  
        popup=str(row['count']), 
        color=get_color(row['count']),
        fill=True,
        fill_color=get_color(row['count'])
    ).add_to(marker_cluster)

# Save or show the map
map_la.save('LA_Crime_Cluster_Map.html')

# Demographic

In [ ]:
def GetAgeCategory(age):
    if age<= 0: return 'Unknown'
    elif age<=4: return 'Baby (0, 4]'
    elif age<=12: return 'Child [5, 12]'
    elif age<=19: return 'Teen [13, 19]'
    elif age<=39: return 'Adult [20, 39]'
    elif age<=59: return 'Middle Age (39, 59]'
    else: return 'Senior [60, )'

grouped_descent_code_mapping = {
    "A": "Asian",
    "B": "Black",
    "C": "Asian",
    "D": "Asian",
    "F": "Asian",
    "G": "Pacific Islander",
    "H": "Hispanic/Latin/Mexican",
    "I": "Native American",
    "J": "Asian",
    "K": "Asian",
    "L": "Asian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Pacific Islander",
    "U": "Pacific Islander",
    "V": "Asian",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian",
    "-": "Unknown"
}

gender_mapping = {
    'X': 'Unknown',
    'H': 'Unknown',
    '-': 'Unknown',
    'F': 'Female',
    'M': 'Male' 
}

pd_crime['VictAge'] = pd_crime['Vict Age'].apply(lambda x: GetAgeCategory(x))
pd_crime['VictSex'] = pd_crime['Vict Sex'].map(gender_mapping).fillna('Unknown')
pd_crime['VictDescent'] = pd_crime['Vict Descent'].map(grouped_descent_code_mapping).fillna('Unknown')

data = {
    "VictDescent": [
        "Other", "Hispanic/Latin/Mexican", "Black", "White", 
        "Asian", "Native American", "Pacific Islander"
    ],
    "pect_pop": [
        3.6,  # Approximation for 'Other'
        48.621,  # Hispanic/Latin/Mexican
        7.58,    # Black
        25.2,  # White
        14.6,  # Asian
        0.188,     # Native American
        0.2      # Pacific Islander
    ]
}

total_population = 9663345  
percentages = [3.6, 48.621, 7.58, 25.2, 14.6, 0.188, 0.2]

populations = [round(p / 100 * total_population) for p in percentages]

data_population = {
    "VictDescent": [
        "Other", "Hispanic/Latin/Mexican", "Black", "White", 
        "Asian", "Native American", "Pacific Islander"
    ],
    "Population": populations
}

pd_pop = pd.DataFrame(data_population)

In [ ]:
fig = px.histogram(pd_crime, x='year', color='VictSex', barmode='group',
                   labels={'year': '', 'VictSex': 'Victim'},
                   title='<b>Crime Trend by Victim Gender by Year</b>',
                   category_orders={'VictSex': ['Male', 'Female', 'Unknown']},
                   width=600, height=400) 

fig.write_image('image/age.png')
fig.show()

In [ ]:
ii_filter = (pd_crime['Vict Age'].notnull()) \
& (pd_crime['VictAge'] != 'Unknown') \
& (pd_crime['VictSex'] != 'Unknown') 
pd_crime_filtered = pd_crime[ii_filter]

In [ ]:
fig = go.Figure()
pd_crime_filtered['year'] = pd_crime_filtered['year'].astype(int)
sorted_years = sorted(pd.unique(pd_crime_filtered['year']))

for year in sorted_years:
    showlegend = bool(year == sorted_years[0])  
    fig.add_trace(go.Violin(x=pd_crime_filtered['year'][(pd_crime_filtered['VictSex'] == 'Male') & 
                                                        (pd_crime_filtered['year'] == year)],
                            y=pd_crime_filtered['Vict Age'][(pd_crime_filtered['VictSex'] == 'Male') & 
                                                            (pd_crime_filtered['year'] == year)],
                            legendgroup='Male', scalegroup='Male', name='Male',
                            side='negative',
                            line_color='lightseagreen',
                            showlegend=showlegend))  

    fig.add_trace(go.Violin(x=pd_crime_filtered['year'][(pd_crime_filtered['VictSex'] == 'Female') & 
                                                        (pd_crime_filtered['year'] == year)],
                            y=pd_crime_filtered['Vict Age'][(pd_crime_filtered['VictSex'] == 'Female') & 
                                                            (pd_crime_filtered['year'] == year)],
                            legendgroup='Female', scalegroup='Female', name='Female',
                            side='positive',
                            line_color='mediumpurple',
                            showlegend=showlegend)) 

fig.update_traces(meanline_visible=True)

fig.update_layout(
    violingap=0, 
    violinmode='overlay',
    title_text="<b>Distribution of Victim Age by Gender and Year</b>",
    yaxis_title="Victim Age",
    xaxis_title="",
    xaxis=dict(range=[min(sorted_years) - 0.5, max(sorted_years) + 0.5])
    
)

fig.write_image('image/violin.png')
fig.show()


In [ ]:
import pandas as pd
from scipy import stats
import statsmodels.api as sm

male_ages = pd_crime_filtered[pd_crime_filtered['VictSex'] == 'Male']['Vict Age'].values
female_ages = pd_crime_filtered[pd_crime_filtered['VictSex'] == 'Female']['Vict Age'].values

statistic, p_value = stats.ks_2samp(male_ages, female_ages)

print(f"KS statistic: {statistic}")
print(f"P-value: {p_value}")


if ks_p_value < 0.05:
    print("Reject null hypothesis - distributions are different")
else:
    print("Fail to reject null hypothesis - no significant difference found")

z_stat, z_p_value = sm.stats.ztest(male_ages, female_ages, alternative='larger')

print(f"Z-test statistic: {z_stat}")
print(f"Z-test P-value: {z_p_value}")
if z_p_value < 0.05:
    print("Reject null hypothesis - mean age of males is greater")
else:
    print("Fail to reject null hypothesis - no significant difference in mean ages")

In [ ]:
fig = px.histogram(pd_crime, x='VictDescent', color='VictSex', barmode='group',
                   labels={'VictDescent': ''},
                   category_orders={
                       'VictSex': ['Male', 'Female', 'Unknown'],
                       'VictDescent': ["Hispanic/Latin/Mexican", "Black", "White", 
                                       "Asian", "Native American", "Pacific Islander", "Other", "Unknown"]
                   },
                   title='<b>Crime Analysis by Victim Descent and Gender</b>')

fig.update_yaxes(title_text='Crime Reports')

fig.update_xaxes(categoryorder='array', categoryarray=["Hispanic/Latin/Mexican", "Black", "White", 
                                                       "Asian", "Native American", "Pacific Islander", "Other", "Unknown"])
fig.write_image('image/descent_gender.png')
fig.show()


In [ ]:
pd_desc = pd_crime['VictDescent'].value_counts().reset_index()
pd_desc = pd_desc.merge(pd_pop, on='VictDescent', how='left')
pd_desc['crime_rate'] = pd_desc['count'] / (3 * pd_desc['Population'])

In [ ]:
pd_desc = pd_crime['VictDescent'].value_counts().reset_index()
pd_desc.columns = ['VictDescent', 'count']
pd_desc = pd_desc.merge(pd_pop, on='VictDescent', how='left')
pd_desc['crime_rate'] = pd_desc['count'] / (3 * pd_desc['Population'])
pd_desc_filtered = pd_desc[pd_desc['VictDescent'] != 'Unknown']


fig = px.bar(pd_desc_filtered, x='crime_rate', y='VictDescent', orientation='h',
             labels={'crime_rate': 'Crime Rate', 'VictDescent': ''},
             title='<b>Crime Rate by Victim Descent Per Year</b>',
             category_orders={
                 'VictDescent': ["Hispanic/Latin/Mexican", "Black", "White", 
                                 "Asian", "Native American", "Pacific Islander", "Other", "Unknown"]
             },
             text='crime_rate') 

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.write_image('image/descent_cr.png')
fig.show()

In [ ]:

fig = px.box(pd_crime_filtered, x='VictDescent', y='Vict Age', color='VictDescent',
             labels={'VictDescent': ''},
             title='<b>Age Distribution by Ethnic Groups</b>',
             width=800, height=500)

fig.update_xaxes(categoryorder='array', categoryarray=["Hispanic/Latin/Mexican", "Black", "White", 
                                                       "Asian", "Native American", "Pacific Islander", "Other", "Unknown"])
fig.update_yaxes(title_text='Victim Age')
fig.update_layout(showlegend=False)

fig.write_image('image/boxplot.png')
fig.show()


In [ ]:
crime_mapping_inverted = {
    "VEHICLE - STOLEN": "Vehicle Crimes",
    "BIKE - STOLEN": "Vehicle Crimes",
    "THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)": "Vehicle Crimes",
    "THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)": "Vehicle Crimes",
    "BURGLARY FROM VEHICLE": "Property Theft",
    "SHOPLIFTING-GRAND THEFT ($950.01 & OVER)": "Property Theft",
    "SHOPLIFTING - PETTY THEFT ($950 & UNDER)": "Property Theft",
    "THEFT-GRAND ($950.01 & OVER)EXCEPT GUNS, FOWL, LIVESTOCK, PRODUCE": "Property Theft",
    "THEFT PLAIN - PETTY ($950 & UNDER)": "Property Theft",
    "THEFT PLAIN - ATTEMPT": "Property Theft",
    "BURGLARY": "Property Theft",
    "EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)": "Property Theft",
    "BUNCO, GRAND THEFT": "Property Theft",
    "THEFT OF IDENTITY": "Financial and Identity Fraud",
    "DOCUMENT FORGERY / STOLEN FELONY": "Financial and Identity Fraud",
    "UNAUTHORIZED COMPUTER ACCESS": "Financial and Identity Fraud",
    "SEX OFFENDER REGISTRANT OUT OF COMPLIANCE": "Financial and Identity Fraud",
    "BATTERY - SIMPLE ASSAULT": "Physical Assaults",
    "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT": "Physical Assaults",
    "INTIMATE PARTNER - SIMPLE ASSAULT": "Physical Assaults",
    "INTIMATE PARTNER - AGGRAVATED ASSAULT": "Physical Assaults",
    "BATTERY WITH SEXUAL CONTACT": "Physical Assaults",
    "SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH": "Sexual Assaults",
    "SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ)": "Sexual Assaults",
    "RAPE, FORCIBLE": "Sexual Assaults",
    "ORAL COPULATION": "Sexual Assaults",
    "LETTERS, LEWD - TELEPHONE CALLS, LEWD": "Harassment",
    "CHILD ANNOYING (17YRS & UNDER)": "Harassment",
    "LEWD CONDUCT": "Harassment",
    "VIOLATION OF RESTRAINING ORDER": "Harassment",
    "CRIMINAL THREATS - NO WEAPON DISPLAYED": "Threats",
    "VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)": "Vandalism",
    "BRANDISH WEAPON": "Vandalism",
    "CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)": "Miscellaneous",
    "OTHER MISCELLANEOUS CRIME": "Miscellaneous"
}

# Example usage
pd_crime['crime_type'] = pd_crime['Crm Cd Desc'].map(crime_mapping_inverted)
pd_crime['crime_type'].value_counts()

In [ ]:
crime_counts = pd_crime['crime_type'].value_counts()
fig = px.pie(values=crime_counts.values, names=crime_counts.index, 
             title='<b>Crime Type Distribution</b>',
             hole=0.3, width=600, height=400)  

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.write_image('image/crime_type.png')
fig.show()


In [ ]:
crime_sex_grouped = pd_crime.groupby(['crime_type', 'VictSex']).size().reset_index(name='counts')

total_counts_by_crime = crime_sex_grouped.groupby('crime_type')['counts'].transform('sum')
crime_sex_grouped['percentage'] = (crime_sex_grouped['counts'] / total_counts_by_crime) * 100
crime_sex_grouped['percentage_text'] = crime_sex_grouped['percentage'].apply(lambda x: f"{x:.2f}%")


fig = px.bar(crime_sex_grouped, x='percentage', y='crime_type', color='VictSex',
             title='<b>Gender Distribution of Crime Types</b>',
             labels={'percentage': 'Percentage', 'crime_type': '', 'VictSex': 'Gender'},
             orientation='h',
             opacity=0.9,
             text='percentage_text')  

fig.update_layout(barmode='stack', 
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  xaxis_title="Percentage (%)",
                  yaxis_title="",
                  yaxis={'categoryorder': 'total ascending'},
                  width=800, height=400)  
fig.write_image('image/gender_dist.png')
fig.show()


In [ ]:

fig = px.box(pd_crime_filtered, x='crime_type', y='Vict Age', color='crime_type',
             labels={'crime_type': ''},
             title='<b>Age Distribution by Crime Type</b>',
             width=800, height=500)

fig.update_yaxes(title_text='Victim Age')
fig.update_layout(showlegend=False)
fig.write_image('image/boxplot_crime_type.png')
fig.show()
